<a href="https://colab.research.google.com/github/timothyow/News_monitoring_agent/blob/main/News_Monitoring_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 0: Upload Word Document
Use either Option A (Jupyter) or Option B (Google Colab) to upload your `.docx` file.

In [ ]:
# ✅ Option A: Jupyter Notebook Upload
try:
    import ipywidgets as widgets
    from IPython.display import display

    upload_widget = widgets.FileUpload(accept='.docx', multiple=False)
    display(upload_widget)
except:
    print("ipywidgets not available or not in Jupyter environment")

In [ ]:
# Save uploaded file to disk (if using Option A)
if 'upload_widget' in locals() and upload_widget.value:
    uploaded_file = list(upload_widget.value.values())[0]
    with open("uploaded_doc.docx", "wb") as f:
        f.write(uploaded_file["content"])
    DOC_PATH = "uploaded_doc.docx"

In [1]:
# Option B: Google Colab Upload
try:
    from google.colab import files
    uploaded = files.upload()
    DOC_PATH = list(uploaded.keys())[0]
except:
    print("google.colab module not available or not in Colab environment")

Saving 2025_05_30_Opco_reports_full.docx to 2025_05_30_Opco_reports_full.docx


# Install dependencies

In [2]:
!pip install langchain==0.3.14
!pip install langchain-openai==0.3.0
!pip install langchain-community==0.3.14
!pip install langgraph==0.2.64
!pip install docx2txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 87.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.42
    Uninstalling langsmith-0.3.42:
      Successfully uninstalled langsmith-0.3.42
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy

# News Monitoring Agent
This notebook uses LangGraph to:
1. Read a Word document of country-level news updates
2. Extract and summarize per-country insights
3. Synthesize a regional summary
4. Generate a polished email to the Asia President

In [3]:
# Imports & Setup
from langchain.document_loaders import Docx2txtLoader
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict
from pydantic import BaseModel
import logging

logging.basicConfig(level=logging.INFO)

# Open AI API key and environment variables

In [6]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


In [7]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

In [8]:
# Define State
class GraphState(TypedDict):
    document_path: str
    document_text: str
    country_updates: str
    regional_summary: str
    email_draft: str

In [9]:
# Config
DOC_PATH = "2025_05_30_Opco_reports_full.docx"
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [10]:
# Prompt Templates
READER_PROMPT = """
You are an expert BCG consultant. Below is a Word document with updates for several countries. Some text is in paragraphs, some in tables.
Extract and summarize each country’s key updates.

Document:
----
{document_text}
----

Return format:
Country: <country name>
Summary: <executive summary>
"""

SUMMARY_PROMPT = """
You are a strategy analyst. Based on the following country updates, summarize the key regional themes and strategic takeaways.

Country Updates:
----
{country_updates}
----

Return a 3-paragraph summary with insights across countries.
"""

EMAIL_PROMPT = """
You are a senior communications manager. Write a professional email regarding news monitoring highlights from our Asia countries, to the Asia President based on the summary below.
The email content is to contain the below two sections:
1.  Key highlights from the news monitoring (indicate the country)
2.  Recommendations
Keep email within 400 words.
Do not include any intro or summary sentence. Do not add extra text.

Summary:
----
{regional_summary}
----

Return a polished email in proper format.
"""

In [11]:
# Setup agents
def reader_agent(state: GraphState) -> GraphState:
    path = state["document_path"]
    loader = Docx2txtLoader(path)
    docs = loader.load()
    text = docs[0].page_content
    prompt = READER_PROMPT.format(document_text=text)
    response = llm.invoke(prompt)
    return {
        "document_text": text,
        "country_updates": response.content,
        "regional_summary": "",
        "email_draft": ""
    }

In [12]:
def summary_agent(state: GraphState) -> GraphState:
    prompt = SUMMARY_PROMPT.format(country_updates=state["country_updates"])
    response = llm.invoke(prompt)
    return {
        **state,
        "regional_summary": response.content
    }

In [13]:
def editor_agent(state: GraphState) -> GraphState:
    prompt = EMAIL_PROMPT.format(regional_summary=state["regional_summary"])
    response = llm.invoke(prompt)
    return {
        **state,
        "email_draft": response.content
    }

In [14]:
# Build LangGraph
READER, SUMMARIZER, EDITOR = "Reader", "Summarizer", "Editor"

builder = StateGraph(GraphState)
builder.add_node(READER, reader_agent)
builder.add_node(SUMMARIZER, summary_agent)
builder.add_node(EDITOR, editor_agent)
builder.set_entry_point(READER)
builder.add_edge(READER, SUMMARIZER)
builder.add_edge(SUMMARIZER, EDITOR)
builder.add_edge(EDITOR, END)
graph = builder.compile()

In [15]:
# Run Graph
initial_state = {
    "document_path": DOC_PATH,
    "document_text": "",
    "country_updates": "",
    "regional_summary": "",
    "email_draft": ""
}

final_state = graph.invoke(initial_state)
print("\n--- Final Email Output ---\n")
print(final_state["email_draft"].strip())


--- Final Email Output ---

Subject: News Monitoring Highlights and Strategic Recommendations for Asia

Dear [Asia President's Name],

1. Key Highlights from the News Monitoring:

- **Indonesia**: The country is making significant strides in agricultural development by enhancing its cattle farming capacity through international partnerships. The Free Nutritious Meal Program is expected to boost the economy. However, challenges such as disease in dairy cattle necessitate strategic partnerships and diversification in dairy products to mitigate risks.

- **Philippines**: Under President Marcos Jr., the Philippines is focusing on improving government services and ensuring policy continuity. The country is experiencing a trade recovery, but agricultural challenges, such as the sugarcane pest infestation, pose risks. Efforts to streamline bureaucracy and promote environmental sustainability are underway, reflecting a regional trend towards efficiency and eco-consciousness.

- **Thailand**: 